# Support Vector Machine Multiclass


### 1) Import Data

In [7]:
#!wget https://dataset-ppm.s3.amazonaws.com/iris.csv
# https://drive.google.com/file/d/1uL6vvfO6fYCi3ilpu7-7cmXqmB8qxEtN
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uL6vvfO6fYCi3ilpu7-7cmXqmB8qxEtN' -O iris.csv

--2024-02-06 23:22:08--  https://docs.google.com/uc?export=download&id=1uL6vvfO6fYCi3ilpu7-7cmXqmB8qxEtN
Resolving docs.google.com (docs.google.com)... 173.194.202.138, 173.194.202.101, 173.194.202.102, ...
Connecting to docs.google.com (docs.google.com)|173.194.202.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1uL6vvfO6fYCi3ilpu7-7cmXqmB8qxEtN&export=download [following]
--2024-02-06 23:22:08--  https://drive.usercontent.google.com/download?id=1uL6vvfO6fYCi3ilpu7-7cmXqmB8qxEtN&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.20.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5107 (5.0K) [application/octet-stream]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   4.99K  --.-KB/s    in 0s   

In [8]:
import numpy as np
import pandas as pd
data = pd.read_csv('iris.csv')

In [9]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


## 2) Splitting Training and Testing Data


In [10]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

In [11]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


In [12]:
label_uji = data_uji.pop('Species')

## 3) Training One VS Rest

In [13]:
def buat_trainingset(dataset):
  trainingset = {}
  kolom_kelas = dataset.columns[-1]
  list_kelas = dataset[kolom_kelas].unique()
  for kelas in list_kelas:
    data_temp = dataset.copy(deep=True)
    data_temp[kolom_kelas] = data_temp[kolom_kelas].map({kelas:1})
    data_temp[kolom_kelas] = data_temp[kolom_kelas].fillna(-1)
    trainingset[kelas] = data_temp
  return trainingset

In [14]:
trainingset = buat_trainingset(data_latih)

In [15]:
print(trainingset)

{'Iris-virginica':       Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  Species
100  101            6.3           3.3            6.0           2.5      1.0
71    72            6.1           2.8            4.0           1.3     -1.0
23    24            5.1           3.3            1.7           0.5     -1.0
16    17            5.4           3.9            1.3           0.4     -1.0
132  133            6.4           2.8            5.6           2.2      1.0
..   ...            ...           ...            ...           ...      ...
97    98            6.2           2.9            4.3           1.3     -1.0
70    71            5.9           3.2            4.8           1.8     -1.0
17    18            5.1           3.5            1.4           0.3     -1.0
91    92            6.1           3.0            4.6           1.4     -1.0
3      4            4.6           3.1            1.5           0.2     -1.0

[120 rows x 6 columns], 'Iris-versicolor':       Id  SepalLengthCm  

## 4) Shaping SVM Binary

In [16]:
def hitung_cost_gradient(W,X,Y,regularization):
  jarak=1-(Y*np.dot(X,W))
  dw=np.zeros(len(W))
  if max(0,jarak)==0:
    di=W
  else:
    di=W-(regularization*Y*X)
  dw+=di
  return dw

In [17]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate=0.000001,max_epoch=1000,regularization=10000):
  data_latih=data_latih.to_numpy()
  label_latih=label_latih.to_numpy()
  bobot=np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y=shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta=hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot=bobot-(learning_rate*delta)
  return bobot

## 5) Training

In [18]:
def training(trainingset):
  list_kelas = trainingset.keys()
  w = {}
  for kelas in list_kelas:
    data_latih = trainingset[kelas]
    label_latih = data_latih.pop(data_latih.columns[-1])
    w[kelas] = sgd(data_latih,label_latih)
  return w

In [19]:
W = training(trainingset)

In [20]:
print(W)

{'Iris-virginica': array([  2.00827248, -43.15207988, -33.68504949,  24.81181725,
        23.09933281]), 'Iris-versicolor': array([ -0.62480967,  12.34625995, -27.17770752,  25.48656234,
        -7.8715309 ]), 'Iris-setosa': array([-0.65628581,  4.7707009 ,  5.19519387, -2.70132644, -1.41139755])}


## 6) Binary Testing

In [21]:
def testing(W,data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi=np.sign(np.dot(W,data_uji.to_numpy()[i]))
    prediksi=np.append(prediksi,y_prediksi)
  return prediksi

In [22]:
def testing_onevsrest(W,data_uji):
  kelas_prediksi = [""]*len(data_uji)
  for i in range(data_uji.shape[0]):
    for key, value in W.items():
      y_prediksi= np.sign(np.dot(value,data_uji.to_numpy()[i]))
      if y_prediksi==1:
        kelas_prediksi[i]=key
  return kelas_prediksi

In [23]:
prediksi = testing_onevsrest(W,data_uji)

In [24]:
print(prediksi)

['Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa']


In [25]:
print(sum(prediksi==label_uji), 'data')

22 data
